In [1]:
import pandas as pd
import json

!pip install folium
import folium
from folium import plugins

     |████████████████████████████████| 102kB 7.1MB/s ta 0:00:011


In [2]:
#hiden cell
CLIENT_ID = '***************' # your Foursquare ID
CLIENT_SECRET = '********************' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: *************
CLIENT_SECRET:*************


In [17]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "categoryId": "4bf58dd8d48988d114951735",#category ID Bookstore
    "near": "Toulouse",
    "radius": 4000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [18]:
d = data.json()["response"]
d.keys()

dict_keys(['geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [19]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Toulouse', 'Toulouse')

In [20]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 43.61182128582971, 'lng': 1.4478726804999997},
  'sw': {'lat': 43.60150557757607, 'lng': 1.4410379295000002}},
 7)

In [21]:
d["geocode"]

{'what': '',
 'where': 'toulouse',
 'center': {'lat': 43.60426, 'lng': 1.44367},
 'displayString': 'Toulouse, France',
 'cc': 'FR',
 'geometry': {'bounds': {'ne': {'lat': 43.66907994633145,
    'lng': 1.5156179631414892},
   'sw': {'lat': 43.53262007719572, 'lng': 1.2822381092561068}}},
 'slug': 'toulouse-france',
 'longId': '72057594040900251'}

In [22]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [23]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [24]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 7


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4bc87fb115a7ef3b7d297bda',
  'name': 'Ombres Blanches',
  'location': {'address': '50, rue Gambetta',
   'lat': 43.60314614926247,
   'lng': 1.4425149834433009,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.60314614926247,
     'lng': 1.4425149834433009}],
   'postalCode': '31000',
   'cc': 'FR',
   'city': 'Toulouse',
   'state': 'Midi-Pyrénées',
   'country': 'France',
   'formattedAddress': ['50, rue Gambetta', '31000 Toulouse', 'France']},
  'categories': [{'id': '4bf58dd8d48988d114951735',
    'name': 'Bookstore',
    'pluralName': 'Bookstores',
    'shortName': 'Bookstore',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/bookstore_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4bc87fb115a7ef3b7d297bda-0'}

In [28]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i Bookstore" % len(df))
df.head()

{'address': '50, rue Gambetta', 'lat': 43.60314614926247, 'lng': 1.4425149834433009, 'labeledLatLngs': [{'label': 'display', 'lat': 43.60314614926247, 'lng': 1.4425149834433009}], 'postalCode': '31000', 'cc': 'FR', 'city': 'Toulouse', 'state': 'Midi-Pyrénées', 'country': 'France', 'formattedAddress': ['50, rue Gambetta', '31000 Toulouse', 'France']}
{'address': '3 Rue du Taur', 'lat': 43.60486219648067, 'lng': 1.4428243550227642, 'labeledLatLngs': [{'label': 'display', 'lat': 43.60486219648067, 'lng': 1.4428243550227642}], 'postalCode': '31000', 'cc': 'FR', 'city': 'Toulouse', 'state': 'Midi-Pyrénées', 'country': 'France', 'formattedAddress': ['3 Rue du Taur', '31000 Toulouse', 'France']}
{'address': '10 rue Baour Lormian', 'lat': 43.602985, 'lng': 1.44479, 'labeledLatLngs': [{'label': 'display', 'lat': 43.602985, 'lng': 1.44479}], 'postalCode': '31000', 'cc': 'FR', 'city': 'Toulouse', 'state': 'Languedoc-Roussillon-Midi-Pyrénées', 'country': 'France', 'formattedAddress': ['10 rue Baou

,uid,name,shortname,address,postalcode,lat,lng
0,4bc87fb115a7ef3b7d297bda,Ombres Blanches,Bookstore,,31000,43.603146,1.442515
1,4be96c46fbbb9c74153f1e8e,Gibert Joseph,Bookstore,,31000,43.604862,1.442824
2,59c4ec2ac4df1d02e45f965b,la phim édition,Bookstore,,31000,43.602985,1.444790
3,4d848c3ff1e56ea844b5768a,Bédéciné,Bookstore,,31000,43.604739,1.441814
4,52331ab611d2031aaf8a89d5,Librairie Des Suds,Bookstore,,31000,43.602514,1.441349


In [29]:
toulouse_center = d["geocode"]["center"]
toulouse_center

{'lat': 43.60426, 'lng': 1.44367}

In [30]:
from folium import plugins


map_toulouse = folium.Map(location=[43.60426, 1.44367], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_toulouse)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_toulouse.add_child(plugins.HeatMap(hm_data))

map_toulouse